#  DENGELI ÖRNEKLEME: RF vs XGBoost


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

BASE_DIR = Path("/content/drive/MyDrive/SA_CropType_SourceCoop")
OUT_DIR = BASE_DIR / "outputs"
META_DIR = OUT_DIR / "meta"
FIG_DIR = OUT_DIR / "figures"
TAB_DIR = OUT_DIR / "tables"

for d in [OUT_DIR, META_DIR, FIG_DIR, TAB_DIR]:
    d.mkdir(parents=True, exist_ok=True)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix
)

!pip install xgboost -q
import xgboost as xgb
import time
from datetime import datetime
import json

print("✅ Kurulum tamam!")

✅ Kurulum tamam!


##  ÖNCEKİ VERİYİ YÜKLE (100 tile sonuçları)

In [ ]:
# Önceki notebook'tan oluşan veriyi yükle
# NOT: Eğer 100 tile notebook'u çalıştırmadıysanız, önce onu çalıştırın!

print("📂 Önceki veri aranıyor...\n")

# Dataset info
info_path = TAB_DIR / 'dataset_info_100tiles.json'
if info_path.exists():
    with open(info_path, 'r') as f:
        prev_info = json.load(f)

    print("✅ Önceki veri bulundu!")
    print(f"\n📊 ÖNCEKİ SONUÇLAR (100 tile):")
    print(f"  Toplam örnek: {prev_info['total_samples']:,}")
    print(f"  Özellik sayısı: {prev_info['n_features']}")
    print(f"  Dengesizlik: {prev_info['imbalance_ratio']:.2f}x")
    print(f"\n  Sınıf dağılımı:")
    for crop, count in prev_info['class_distribution'].items():
        pct = count / prev_info['total_samples'] * 100
        print(f"    {crop:30s}: {count:5,} ({pct:5.1f}%)")
else:
    print("⚠️ Önceki veri bulunamadı!")
    print("   Önce 10_OPTIMIZED_100tiles_15dates.ipynb çalıştırın.")
    print("\n   Şimdilik varsayılan değerlerle devam ediyoruz...")
    prev_info = None

📂 Önceki veri aranıyor...

✅ Önceki veri bulundu!

📊 ÖNCEKİ SONUÇLAR (100 tile):
  Toplam örnek: 8,228
  Özellik sayısı: 22
  Dengesizlik: 4.61x

  Sınıf dağılımı:
    Lucerne/Medics                : 1,043 ( 12.7%)
    Planted pastures (perennial)  : 1,326 ( 16.1%)
    Wheat                         : 1,050 ( 12.8%)
    Wine grapes                   : 4,809 ( 58.4%)


###  DENGELI ÖRNEKLEME

In [ ]:
print("⚖️ Dengeli örnekleme başlıyor...\n")

# TFC yükle
tfc_selected = pd.read_csv(META_DIR / "tfc_selected_top100.csv")

TARGET_CROPS = [
    "Wine grapes",
    "Wheat",
    "Planted pastures (perennial)",
    "Lucerne/Medics"
]

print("📊 MEVCUT DAĞILIM:")
print("="*60)
for crop in TARGET_CROPS:
    count = len(tfc_selected[tfc_selected['crop'] == crop])
    print(f"  {crop:30s}: {count:5,}")
print("="*60)

# En az örnek sayısını bul
min_samples = tfc_selected['crop'].value_counts().min()
print(f"\n📌 En az örnek: {min_samples:,}\n")

# DENGELI ÖRNEKLEME STRATEJİSİ
# Seçenek 1: Min sample (en az örneği olan sınıfın sayısı)
# Seçenek 2: Sabit sayı (örn: 1000)

# Her sınıftan en fazla kaç tane alabiliriz?
max_possible = min_samples

# Hedef: 1000 (eğer yeterli veri varsa)
TARGET_PER_CLASS = min(1000, max_possible)

print(f"🎯 HEDEF: Her sınıftan {TARGET_PER_CLASS:,} örnek\n")

# Her sınıftan rastgele örnekle
balanced_samples = []

for crop in TARGET_CROPS:
    crop_data = tfc_selected[tfc_selected['crop'] == crop]

    if len(crop_data) >= TARGET_PER_CLASS:
        # Rastgele örnekle
        sampled = crop_data.sample(n=TARGET_PER_CLASS, random_state=42)
        balanced_samples.append(sampled)
        print(f"  ✅ {crop:30s}: {len(sampled):,} örnek seçildi")
    else:
        # Hepsini al (yeterli değilse)
        balanced_samples.append(crop_data)
        print(f"  ⚠️ {crop:30s}: {len(crop_data):,} örnek (hepsi alındı)")

# Birleştir
balanced_tfc = pd.concat(balanced_samples, ignore_index=True)

# Karıştır
balanced_tfc = balanced_tfc.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"\n✅ DENGELI VERİ SETİ OLUŞTURULDU!")
print(f"  Toplam: {len(balanced_tfc):,} örnek")
print(f"  Her sınıf: ~{TARGET_PER_CLASS:,} örnek")

print(f"\n📊 YENİ DAĞILIM:")
print("="*60)
for crop in TARGET_CROPS:
    count = len(balanced_tfc[balanced_tfc['crop'] == crop])
    pct = count / len(balanced_tfc) * 100
    print(f"  {crop:30s}: {count:5,} ({pct:5.1f}%)")
print("="*60)

# Dengelilik oranı
counts = balanced_tfc['crop'].value_counts()
imbalance = counts.max() / counts.min()
print(f"\n⚖️ Dengelilik oranı: {imbalance:.2f}x")

if imbalance < 1.2:
    print("   ✅ MÜKEMMEL DENGE!")
elif imbalance < 2:
    print("   ✅ İYİ DENGE")
else:
    print("   ⚠️ Hala dengesiz")

# Kaydet
balanced_tfc.to_csv(META_DIR / "tfc_balanced_1000_per_class.csv", index=False)
print(f"\n💾 Kaydedildi: tfc_balanced_1000_per_class.csv")

⚖️ Dengeli örnekleme başlıyor...

📊 MEVCUT DAĞILIM:
  Wine grapes                   : 4,809
  Wheat                         : 1,050
  Planted pastures (perennial)  : 1,326
  Lucerne/Medics                : 1,043

📌 En az örnek: 1,043

🎯 HEDEF: Her sınıftan 1,000 örnek

  ✅ Wine grapes                   : 1,000 örnek seçildi
  ✅ Wheat                         : 1,000 örnek seçildi
  ✅ Planted pastures (perennial)  : 1,000 örnek seçildi
  ✅ Lucerne/Medics                : 1,000 örnek seçildi

✅ DENGELI VERİ SETİ OLUŞTURULDU!
  Toplam: 4,000 örnek
  Her sınıf: ~1,000 örnek

📊 YENİ DAĞILIM:
  Wine grapes                   : 1,000 ( 25.0%)
  Wheat                         : 1,000 ( 25.0%)
  Planted pastures (perennial)  : 1,000 ( 25.0%)
  Lucerne/Medics                : 1,000 ( 25.0%)

⚖️ Dengelilik oranı: 1.00x
   ✅ MÜKEMMEL DENGE!

💾 Kaydedildi: tfc_balanced_1000_per_class.csv


##  ÖZELLİK ÇIKARIMI (Dengeli Veri)

In [ ]:
import rasterio

TRAIN_DIR = BASE_DIR / "train"
LABELS_DIR = TRAIN_DIR / "labels"
S2_DIR = TRAIN_DIR / "imagery" / "s2"

def smart_date_selection(tile_dir, max_dates=15):
    """Akıllı tarih seçimi (aylara dağıtılmış)"""
    all_dates = sorted([d for d in tile_dir.iterdir() if d.is_dir()])
    if len(all_dates) == 0:
        return [], {}

    date_info = []
    for date_folder in all_dates:
        try:
            parts = date_folder.name.split('_')
            if len(parts) >= 3:
                year = int(parts[0])
                month = int(parts[1])
                date_info.append({
                    'path': date_folder,
                    'year_month': f"{year}_{month:02d}"
                })
        except:
            continue

    if len(date_info) == 0:
        return all_dates[:max_dates], {}

    df = pd.DataFrame(date_info)
    monthly_counts = df.groupby('year_month').size()
    n_months = len(monthly_counts)
    dates_per_month = max(1, max_dates // n_months)

    selected_dates = []
    for year_month, group in df.groupby('year_month'):
        n_select = min(dates_per_month, len(group))
        if len(group) > n_select:
            indices = np.linspace(0, len(group)-1, n_select, dtype=int)
            selected = group.iloc[indices]
        else:
            selected = group
        selected_dates.extend(selected['path'].tolist())

    if len(selected_dates) < max_dates:
        remaining = max_dates - len(selected_dates)
        for year_month in monthly_counts.nlargest(remaining).index:
            group = df[df['year_month'] == year_month]
            unselected = [p for p in group['path'] if p not in selected_dates]
            if unselected:
                selected_dates.append(unselected[0])
                if len(selected_dates) >= max_dates:
                    break

    selected_dates = sorted(selected_dates, key=lambda x: x.name)[:max_dates]
    selected_df = df[df['path'].isin(selected_dates)]
    distribution = selected_df.groupby('year_month').size().to_dict()

    return selected_dates, distribution

def extract_features(tile_id, field_id, s2_dir, labels_dir, num_dates=15):
    """Özellik çıkarımı (akıllı tarih seçimi ile)"""
    features = {}

    try:
        label_path = labels_dir / f"{tile_id}_field_ids.tif"
        if not label_path.exists():
            return None

        with rasterio.open(label_path) as src:
            label_mask = src.read(1)

        field_mask = (label_mask == field_id)
        if field_mask.sum() == 0:
            return None

        tile_dir = s2_dir / str(tile_id)
        if not tile_dir.exists():
            return None

        date_folders, _ = smart_date_selection(tile_dir, max_dates=num_dates)
        if len(date_folders) == 0:
            return None

        band_map = {'B2': 'B02', 'B3': 'B03', 'B4': 'B04', 'B8': 'B08'}
        band_values = {name: [] for name in band_map.keys()}

        for date_folder in date_folders:
            date_str = date_folder.name
            for band_short, band_full in band_map.items():
                band_file = date_folder / f"{tile_id}_{date_str}_{band_full}_10m.tif"
                if band_file.exists():
                    with rasterio.open(band_file) as src:
                        band_data = src.read(1)
                    field_vals = band_data[field_mask]
                    field_vals = field_vals[field_vals > 0]
                    if len(field_vals) > field_mask.sum() * 0.3:
                        band_values[band_short].extend(field_vals.tolist())

        for band, vals in band_values.items():
            if len(vals) > 0:
                features[f'{band}_mean'] = np.mean(vals)
                features[f'{band}_std'] = np.std(vals)
                features[f'{band}_min'] = np.min(vals)
                features[f'{band}_max'] = np.max(vals)
                features[f'{band}_median'] = np.median(vals)
            else:
                return None

        if 'B8_mean' in features and 'B4_mean' in features:
            nir = features['B8_mean']
            red = features['B4_mean']
            features['NDVI'] = (nir - red) / (nir + red) if (nir + red) > 0 else 0

        if all(k in features for k in ['B8_mean', 'B4_mean', 'B2_mean']):
            nir = features['B8_mean']
            red = features['B4_mean']
            blue = features['B2_mean']
            denom = nir + 6*red - 7.5*blue + 1
            features['EVI'] = 2.5 * (nir - red) / denom if denom != 0 else 0

        return features
    except:
        return None

print("✅ Özellik fonksiyonları hazır!")

✅ Özellik fonksiyonları hazır!


In [ ]:
print(f"🚀 Özellik çıkarımı başlıyor (DENGELI VERİ)...")
print(f"📊 İşlenecek: {len(balanced_tfc):,} kayıt")
print(f"⏱️  Tahmini süre: ~{len(balanced_tfc)//200} dakika\n")

features_list = []
labels_list = []
failed = 0

for idx, row in tqdm(balanced_tfc.iterrows(), total=len(balanced_tfc), desc="Çıkarım"):
    tile_id = row['tile_id']
    field_id = row['field_id']
    crop = row['crop']

    feats = extract_features(tile_id, field_id, S2_DIR, LABELS_DIR, num_dates=15)

    if feats and len(feats) > 0:
        features_list.append(feats)
        labels_list.append(crop)
    else:
        failed += 1

print(f"\n✅ Tamamlandı!")
print(f"  Başarılı: {len(features_list):,}")
print(f"  Başarısız: {failed}")
print(f"  Başarı oranı: {len(features_list)/(len(features_list)+failed)*100:.1f}%")

🚀 Özellik çıkarımı başlıyor (DENGELI VERİ)...
📊 İşlenecek: 4,000 kayıt
⏱️  Tahmini süre: ~20 dakika



Çıkarım: 100%|██████████| 4000/4000 [39:18<00:00,  1.70it/s]


✅ Tamamlandı!
  Başarılı: 4,000
  Başarısız: 0
  Başarı oranı: 100.0%


## 🎯 VERİ HAZIRLAMA

In [ ]:
X = pd.DataFrame(features_list)
y = np.array(labels_list)

print(f"✅ Dengeli veri hazır:")
print(f"  Shape: {X.shape}")

print(f"\n🌾 FİNAL SINIF DAĞILIMI:")
print("="*60)
unique, counts = np.unique(y, return_counts=True)
for crop, count in zip(unique, counts):
    pct = count/len(y)*100
    print(f"  {crop:30s}: {count:5,} ({pct:5.1f}%)")
print("="*60)

imbalance = counts.max() / counts.min()
print(f"\n⚖️ Dengelilik: {imbalance:.2f}x")

# Class weight gerekli mi?
use_class_weight = imbalance > 1.5
print(f"   Class weight: {'Evet' if use_class_weight else 'Hayır (dengeli)'}")

# Label encoding
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"\n📊 Veri bölme:")
print(f"  Train: {X_train.shape[0]:,}")
print(f"  Test:  {X_test.shape[0]:,}")

✅ Dengeli veri hazır:
  Shape: (4000, 22)

🌾 FİNAL SINIF DAĞILIMI:
  Lucerne/Medics                : 1,000 ( 25.0%)
  Planted pastures (perennial)  : 1,000 ( 25.0%)
  Wheat                         : 1,000 ( 25.0%)
  Wine grapes                   : 1,000 ( 25.0%)

⚖️ Dengelilik: 1.00x
   Class weight: Hayır (dengeli)

📊 Veri bölme:
  Train: 3,200
  Test:  800


## 🌲 RANDOM FOREST (Dengeli Veri)

In [ ]:
print(f"🌲 Random Forest (DENGELI VERİ)...\n")

rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=30,
    class_weight='balanced' if use_class_weight else None,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

start = time.time()
rf_model.fit(X_train, y_train)
rf_time = time.time() - start

y_pred_rf = rf_model.predict(X_test)
rf_acc = accuracy_score(y_test, y_pred_rf)
rf_f1 = f1_score(y_test, y_pred_rf, average='weighted')

print(f"\n✅ Tamamlandı ({rf_time:.2f}s)")
print(f"\n📊 Random Forest (DENGELI):")
print(f"  Accuracy:  {rf_acc:.4f}")
print(f"  F1-Score:  {rf_f1:.4f}")

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


🌲 Random Forest (DENGELI VERİ)...



[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.5s



✅ Tamamlandı (2.10s)

📊 Random Forest (DENGELI):
  Accuracy:  0.6075
  F1-Score:  0.5994


[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    2.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.1s finished


## 🚀 XGBOOST (Dengeli Veri)

In [ ]:
print(f"🚀 XGBoost (DENGELI VERİ)...\n")

xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=8,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1
)

if use_class_weight:
    from sklearn.utils.class_weight import compute_sample_weight
    sample_weights = compute_sample_weight('balanced', y_train)
    start = time.time()
    xgb_model.fit(X_train, y_train, sample_weight=sample_weights)
else:
    start = time.time()
    xgb_model.fit(X_train, y_train)

xgb_time = time.time() - start

y_pred_xgb = xgb_model.predict(X_test)
xgb_acc = accuracy_score(y_test, y_pred_xgb)
xgb_f1 = f1_score(y_test, y_pred_xgb, average='weighted')

print(f"\n✅ Tamamlandı ({xgb_time:.2f}s)")
print(f"\n📊 XGBoost (DENGELI):")
print(f"  Accuracy:  {xgb_acc:.4f}")
print(f"  F1-Score:  {xgb_f1:.4f}")

## 📊 KARŞILAŞTIRMA: Dengesiz vs Dengeli

In [ ]:
print("\n" + "="*80)
print("📊 KAPSAMLI KARŞILAŞTIRMA")
print("="*80)

# Dengesiz sonuçlar (100 tile)
imbalanced_rf = 0.76
imbalanced_xgb = 0.76

comparison = pd.DataFrame({
    'Strateji': ['Dengesiz (8,228)', 'Dengesiz (8,228)', 'Dengeli (~4,000)', 'Dengeli (~4,000)'],
    'Model': ['Random Forest', 'XGBoost', 'Random Forest', 'XGBoost'],
    'Accuracy': [imbalanced_rf, imbalanced_xgb, rf_acc, xgb_acc],
    'F1-Score': [0.73, 0.75, rf_f1, xgb_f1]
})

print("\n" + comparison.to_string(index=False))

# En iyi sonuç
best_idx = comparison['Accuracy'].idxmax()
best = comparison.iloc[best_idx]

print(f"\n🏆 EN İYİ SONUÇ:")
print(f"  Strateji: {best['Strateji']}")
print(f"  Model: {best['Model']}")
print(f"  Accuracy: {best['Accuracy']:.4f}")
print(f"  F1-Score: {best['F1-Score']:.4f}")

# Dengeli vs Dengesiz fark
print(f"\n📈 DENGELI vs DENGESİZ:")
rf_diff = (rf_acc - imbalanced_rf) * 100
xgb_diff = (xgb_acc - imbalanced_xgb) * 100

arrow_rf = "📈" if rf_diff > 0 else "📉"
arrow_xgb = "📈" if xgb_diff > 0 else "📉"

print(f"  {arrow_rf} RF:  {imbalanced_rf:.1%} → {rf_acc:.1%} ({rf_diff:+.1f}%)")
print(f"  {arrow_xgb} XGB: {imbalanced_xgb:.1%} → {xgb_acc:.1%} ({xgb_diff:+.1f}%)")

# Kaydet
comparison.to_csv(TAB_DIR / 'balanced_vs_imbalanced_comparison.csv', index=False)
print(f"\n💾 Karşılaştırma kaydedildi")

In [ ]:
# Confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

cm_rf = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues',
            xticklabels=le.classes_, yticklabels=le.classes_, ax=axes[0])
axes[0].set_title(f'RF - DENGELI (Acc: {rf_acc:.1%})')
axes[0].set_ylabel('Gerçek')
axes[0].set_xlabel('Tahmin')

cm_xgb = confusion_matrix(y_test, y_pred_xgb)
sns.heatmap(cm_xgb, annot=True, fmt='d', cmap='Greens',
            xticklabels=le.classes_, yticklabels=le.classes_, ax=axes[1])
axes[1].set_title(f'XGB - DENGELI (Acc: {xgb_acc:.1%})')
axes[1].set_ylabel('Gerçek')
axes[1].set_xlabel('Tahmin')

plt.tight_layout()
plt.savefig(FIG_DIR / 'confusion_matrices_BALANCED.png', dpi=300)
plt.show()

print("\n🌲 RF - Sınıf Bazında (DENGELI):")
print(classification_report(y_test, y_pred_rf, target_names=le.classes_))

print("\n🚀 XGB - Sınıf Bazında (DENGELI):")
print(classification_report(y_test, y_pred_xgb, target_names=le.classes_))

print("\n💾 Tüm sonuçlar kaydedildi!")